# SQL-with-PySpark

In [1]:
# Making sure to link pyspark to the right Spark folder with findspark
import findspark
import time
from pathlib import Path
from functools import wraps
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
findspark.init('/opt/spark')

In [2]:
! apt-get install -y curl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.24).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [ ]:
! curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

In [ ]:
spark = SparkSession.builder.appName(
    "SQL-with-PySpark"
).config(
    "spark.jars",
    f"{str(Path.cwd())}/sqlite-jdbc-3.34.0.jar"
).config(
    "spark.driver.extraClassPath",
    f"{str(Path.cwd())}/sqlite-jdbc-3.34.0.jar"
).getOrCreate()

In [ ]:
datasets_path = Path("../datasets")
chinook_db = datasets_path / "chinook" / "Chinook_Sqlite.sqlite"

### Tables:

- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track

In [ ]:
df = spark.read.format(
    "jdbc"
).options(
    url=f"jdbc:sqlite:{str(chinook_db)}",
    dbtable="Album",
    driver="org.sqlite.JDBC"
).load()

In [ ]:
df.show()